# Data Preprocessing for Credit Score Dataset

This notebook demonstrates the preprocessing steps for a credit score dataset. 
The steps include cleaning categorical and numerical fields, handling missing values, encoding categorical variables, and normalizing the data.

Import necessary libraries and append system path for module access

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pkg_resources
from typing import Tuple
import scipy.stats as stats

Functions used

In [3]:
# Function to fill missing values with the mode of each group
def fill_missing_with_group_mode(df, groupby, column):
    mode_per_group = df.groupby(groupby)[column].transform(lambda x: x.mode().iat[0])
    df[column] = df[column].fillna(mode_per_group)

# Function to clean categorical fields
def clean_categorical_field(df, groupby, column, replace_value=None):
    if replace_value is not None:
        df[column] = df[column].replace(replace_value, np.nan)
    fill_missing_with_group_mode(df, groupby, column)

# Function to handle outliers and null values
def fix_inconsistent_values(df, groupby, column):
    df_dropped = df[df[column].notna()].groupby(groupby)[column].apply(list)
    x, y = df_dropped.apply(lambda x: stats.mode(x, keepdims=False)).apply([min, max])
    mini, maxi = x[0], y[0]

    col = df[column].apply(lambda x: np.NaN if ((x < mini) | (x > maxi) | (x < 0)) else x)
    mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.NaN)
    df[column] = col.fillna(mode_by_group)
    df[column].fillna(df[column].mean(), inplace=True)

# Function to clean numerical fields
def clean_numerical_field(df, groupby, column, strip=None, datatype=None, replace_value=None):
    if replace_value is not None:
        df[column] = df[column].replace(replace_value, np.nan)
    if df[column].dtype == object and strip is not None:
        df[column] = df[column].str.strip(strip)
    if datatype is not None:
        df[column] = df[column].astype(datatype)
    fix_inconsistent_values(df, groupby, column)

Load dataset

In [ ]:
df_train = pd.read_csv('train.csv')

Cleaning categorical fields

In [ ]:
clean_categorical_field(df_train, 'Customer_ID', 'Name')
clean_categorical_field(df_train, 'Customer_ID', 'SSN', '#F%$D@*&8')
clean_categorical_field(df_train, 'Customer_ID', 'Occupation', '_______')
clean_categorical_field(df_train, 'Customer_ID', 'Credit_Mix', '_')
clean_categorical_field(df_train, 'Customer_ID', 'Payment_Behaviour', '!@9#%8')

# Convert Month to datetime object
df_train['Month'] = pd.to_datetime(df_train['Month'], format='%B').dt.month

# Handle Type of Loan null values
df_train['Type_of_Loan'].replace([np.NaN], 'Not Specified', inplace=True)

Cleaning numerical fields

In [ ]:
clean_numerical_field(df_train, 'Customer_ID', 'Age', strip='_', datatype='int')
clean_numerical_field(df_train, 'Customer_ID', 'Annual_Income', strip='_', datatype='float')
clean_numerical_field(df_train, 'Customer_ID', 'Monthly_Inhand_Salary')
clean_numerical_field(df_train, 'Customer_ID', 'Num_Bank_Accounts')
clean_numerical_field(df_train, 'Customer_ID', 'Num_Credit_Card')
clean_numerical_field(df_train, 'Customer_ID', 'Interest_Rate')
clean_numerical_field(df_train, 'Customer_ID', 'Delay_from_due_date')
clean_numerical_field(df_train, 'Customer_ID', 'Num_of_Delayed_Payment', strip='_', datatype='float')
clean_numerical_field(df_train, 'Customer_ID', 'Changed_Credit_Limit', strip='_', datatype='float', replace_value='_')
clean_numerical_field(df_train, 'Customer_ID', 'Num_Credit_Inquiries')
clean_numerical_field(df_train, 'Customer_ID', 'Outstanding_Debt', strip='_', datatype=float)
clean_numerical_field(df_train, 'Customer_ID', 'Total_EMI_per_month')
clean_numerical_field(df_train, 'Customer_ID', 'Amount_invested_monthly', strip='_', datatype=float)
clean_numerical_field(df_train, 'Customer_ID', 'Monthly_Balance', strip='_', datatype=float, replace_value='__-333333333333333333333333333__')
clean_numerical_field(df_train, 'Customer_ID', 'Num_of_Loan', strip='_', datatype=float)

Convert Credit History Age to months

In [ ]:
# Convert Credit History Age to months
def Month_Converter(val):
    if pd.notnull(val):
        years = int(val.split(' ')[0])
        month = int(val.split(' ')[3])
        return (years * 12) + month
    else:
        return val

df_train['Credit_History_Age'] = df_train['Credit_History_Age'].apply(lambda x: Month_Converter(x)).astype(float)
clean_numerical_field(df_train, 'Customer_ID', 'Credit_History_Age', datatype=float)

In [ ]:
# Check for null values
print(df_train.isna().sum())

Month                       0
Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64


In [ ]:
# Drop unnecessary columns
df_train.drop(['ID', 'Customer_ID', 'Name', 'SSN'], axis=1, inplace=True)

# Label Encoding for categorical columns
categorical_columns = ['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
label_encoder = LabelEncoder()

for column in categorical_columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])




Normalize data

In [ ]:
# Split input and output data
X = df_train.drop('Credit_Score', axis=1)
y = df_train['Credit_Score']

# Normalize data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

Combine X and y and save to a file

In [ ]:
df_processed = pd.DataFrame(X, columns=df_train.drop('Credit_Score', axis=1).columns)
df_processed['Credit_Score'] = y.values
df_processed.to_csv('fico_preprocessed.csv', index=False)